# Train SVM Classifiers 


In [1]:
import random
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from svm_tf_idf import SVM_TF_IDF
from svm_embedding import SVM_Embedding

/home/rowan/miniconda3/envs/defenseinj/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

LABEL2ID = {
    'benign': 0,
    'naive': 1,
    'ignore': 2,
    'escape': 3,
    'combine': 4
}
ID2LABEL = {v:k for k,v in LABEL2ID.items()}

In [3]:
def build_dataset():
    ds = load_dataset('guychuk/open-prompt-injection', split='train')
    texts, labels = [], []
    for row in ds:
        instruction = row['instruction']
        # benign sample
        texts.append(f"Instruction: {instruction}\nUser input: {row['normal_input']}")
        labels.append(LABEL2ID['benign'])
        # attack sample
        texts.append(f"Instruction: {instruction}\nUser input: {row['attack_input']}")
        labels.append(LABEL2ID[row['attack_type']])
    return texts, labels

In [4]:
def split_data(X, y):
    X_train, X_tmp, y_train, y_tmp = train_test_split(
        X, y, test_size=0.3, random_state=SEED, stratify=y
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_tmp, y_tmp, test_size=0.5, random_state=SEED, stratify=y_tmp
    )
    return X_train, X_val, X_test, y_train, y_val, y_test

In [5]:
def evaluate_model(model, X, y, name):
    y_pred = model.predict(X)
    print(f"\n=== {name} ===")
    print("Accuracy:", accuracy_score(y, y_pred))
    print(classification_report(
        y, y_pred, target_names=[ID2LABEL[i] for i in range(len(ID2LABEL))]
    ))

In [6]:
X, y = build_dataset()
X_train, X_val, X_test, y_train, y_val, y_test = split_data(X, y)
print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")

Train: 47040 | Val: 10080 | Test: 10080


In [7]:
tfidf_model = SVM_TF_IDF()
tfidf_model.train(X_train, y_train)
evaluate_model(tfidf_model, X_val, y_val, "TF-IDF SVM (Validation)")
evaluate_model(tfidf_model, X_test, y_test, "TF-IDF SVM (Test)")
tfidf_model.save_model("models/svm_tfidf.pkl")

Model training complete.

=== TF-IDF SVM (Validation) ===
Accuracy: 0.816468253968254
              precision    recall  f1-score   support

      benign       1.00      1.00      1.00      5040
       naive       0.27      0.27      0.27      1260
      ignore       1.00      1.00      1.00      1260
      escape       0.26      0.26      0.26      1260
     combine       1.00      1.00      1.00      1260

    accuracy                           0.82     10080
   macro avg       0.71      0.71      0.71     10080
weighted avg       0.82      0.82      0.82     10080


=== TF-IDF SVM (Test) ===
Accuracy: 0.8178571428571428
              precision    recall  f1-score   support

      benign       1.00      1.00      1.00      5040
       naive       0.27      0.27      0.27      1260
      ignore       1.00      1.00      1.00      1260
      escape       0.27      0.28      0.28      1260
     combine       1.00      1.00      1.00      1260

    accuracy                           0.82

In [9]:
emb_model = SVM_Embedding()
emb_model.train(X_train, y_train)
evaluate_model(emb_model, X_val, y_val, "Embedding SVM (Validation)")
evaluate_model(emb_model, X_test, y_test, "Embedding SVM (Test)")
emb_model.save_model("models/svm_embedding.pkl")

Model training complete.

=== Embedding SVM (Validation) ===
Accuracy: 0.7449404761904762
              precision    recall  f1-score   support

      benign       0.97      1.00      0.98      5040
       naive       0.35      0.29      0.32      1260
      ignore       0.58      0.62      0.60      1260
      escape       0.35      0.30      0.32      1260
     combine       0.66      0.77      0.71      1260

    accuracy                           0.74     10080
   macro avg       0.58      0.59      0.59     10080
weighted avg       0.73      0.74      0.74     10080


=== Embedding SVM (Test) ===
Accuracy: 0.7527777777777778
              precision    recall  f1-score   support

      benign       0.97      0.99      0.98      5040
       naive       0.38      0.30      0.34      1260
      ignore       0.60      0.65      0.63      1260
      escape       0.37      0.32      0.34      1260
     combine       0.66      0.77      0.72      1260

    accuracy                        